##### Udacity Data Engineering Nanodegree

<img alt="" align="right" width="150" height="150" src = "./image/aws_logo.png" title = "aws logo" alt = "aws logo">  
</br>
</br>
</br>

# Project 3 : Data Warehouse

About an ETL pipeline that extracts data from S3, stages them in Redshift, and transforms data into a set of dimensional tables.

## Table
* [Datawarehouse Params](#Datawarehouse-Params)
    * [AWS client](#AWS-client)
* [Explore Data S3](#Explore-Data-S3)
    * [S3 song_data](#S3-song_data)
    * [S3 log_data](#S3-log_data)
* [Explore databases](#Explore-databases)
    * [Explore table staging_songs](#Explore-table-staging_songs)
    * [Explore table staging_events](#Explore-table-staging_events)
* [Queries](#Queries)
    * [1_queries to explore dimTables and Fact table](#1_queries-to-explore-dimTables-and-Fact-table)
    * [2_queries for the analytic team](#2_queries-for-the-analytic-team)
    * [3_Explore Distkey and sortkey](#3_Explore-Distkey-and-sortkey)

---

In [1]:
from time import time
import pandas as pd
import configparser 
import boto3
import json
import os
import io

# Datawarehouse Params

#### Endpoint Explication

* **The cluster endpoint is the URL you can access it from outside the AWS VPN the cluster is connected to**
* [Video Youtube sur Redshift](https://www.youtube.com/watch?v=fc5WPKnbam8&list=PL9ooVrP1hQOFWxRJcGdCot7AgJu29SVV3&index=35) 


In [2]:
# get parameters
config = configparser.ConfigParser() 
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")

DWH_ENDPOINT = config.get("CLUSTER", "HOST")[1:-1] 
DWH_ROLE_ARN = config.get("IAM_ROLE", "ARN")[1:-1] 

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB, DWH_ENDPOINT)

('sparkifyuser',
 'Passw0rd',
 'sparkifydb',
 'dwhcluster.cleupqwcfjq1.us-west-2.redshift.amazonaws.com')

### AWS client

In [3]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name="us-west-2")


# Explore Data S3

In [4]:
udacitybucket=s3.Bucket("udacity-dend")

In [5]:
i=0
for obj in s3.Bucket(name='udacity-dend').objects.all():
    if i > 20:
        break
    i+=1
    print(os.path.join(obj.bucket_name, obj.key))

udacity-dend/data-pipelines/
udacity-dend/data-pipelines/data-pipeline-dataset-creation.zip
udacity-dend/data-pipelines/divvy/partitioned/2018/1/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/10/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/11/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/12/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/2/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/3/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/4/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/5/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/6/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/7/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/8/divvy_trips.csv
udacity-dend/data-pipelines/divvy/partitioned/2018/9/divvy_trips.csv
udacity-dend/data-pipelines/divvy/unpartitioned/divvy_stations_2017.csv
udaci

---

### S3 song_data

In [ ]:
%%time
size_songData = sum(1 for _ in udacitybucket.objects.filter(Prefix="song-data"))
print(size_songData)

In [7]:
%%time
n = 0
for obj in udacitybucket.objects.filter(Prefix="song-data"):
    if n > 5:
        break
    n+=1
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json')
CPU times: user 217 ms, sys: 4.89 ms, total: 222 ms
Wall time: 2.53 s


In [8]:
%%time
for obj in udacitybucket.objects.filter(Prefix='song-data/A/A/A/TRAAAAK128F9'):
    body_song = obj.get()['Body'].read().decode('utf-8')
    print(body_song)

{"song_id": "SOBLFFE12AF72AA5BA", "num_songs": 1, "title": "Scream", "artist_name": "Adelitas Way", "artist_latitude": null, "year": 2009, "duration": 213.9424, "artist_id": "ARJNIUY12298900C91", "artist_longitude": null, "artist_location": ""}
CPU times: user 14.8 ms, sys: 2.55 ms, total: 17.3 ms
Wall time: 527 ms


In [9]:
print(type(body_song))
print(body_song)

<class 'str'>
{"song_id": "SOBLFFE12AF72AA5BA", "num_songs": 1, "title": "Scream", "artist_name": "Adelitas Way", "artist_latitude": null, "year": 2009, "duration": 213.9424, "artist_id": "ARJNIUY12298900C91", "artist_longitude": null, "artist_location": ""}


In [10]:
dic_song = json.loads(body_song)
list(dic_song.keys())

['song_id',
 'num_songs',
 'title',
 'artist_name',
 'artist_latitude',
 'year',
 'duration',
 'artist_id',
 'artist_longitude',
 'artist_location']

In [11]:
my_bucket = s3.Bucket('udacity-dend')
my_bucket.download_file('song_data/A/A/A/TRAAAAK128F9318786.json', 'TRAAAAK128F9318786.json')

In [12]:
with open('TRAAAAK128F9318786.json') as datafile:
    data = json.load(datafile)
df_song = pd.DataFrame(data, index=[0])

In [13]:
print("(df_song shape {}".format(df_song.shape))

(df_song shape (1, 10)


In [14]:
print("(df_song type {}".format(df_song.dtypes))

(df_song type artist_id            object
artist_latitude      object
artist_location      object
artist_longitude     object
artist_name          object
duration            float64
num_songs             int64
song_id              object
title                object
year                  int64
dtype: object


---

### S3 log_data

In [15]:
%%time
size_logData = sum(1 for _ in udacitybucket.objects.filter(Prefix="log_data"))
print(size_logData)

31
CPU times: user 38.1 ms, sys: 1.06 ms, total: 39.2 ms
Wall time: 331 ms


In [16]:
n = 0
for obj in udacitybucket.objects.filter(Prefix="log_data"):
    if n > 5:
        break
    n += 1
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-05-events.json')


In [17]:
for obj in udacitybucket.objects.filter(Prefix='log_data/2018/11/2018-11-30-events.json'):
    body = obj.get()['Body'].read().decode('utf-8')
    line = body[:435]
    print(line)   

{"artist":"Stephen Lynch","auth":"Logged In","firstName":"Jayden","gender":"M","itemInSession":0,"lastName":"Bell","length":182.85669,"level":"free","location":"Dallas-Fort Worth-Arlington, TX","method":"PUT","page":"NextSong","registration":1540991795796.0,"sessionId":829,"song":"Jim Henson's Dead","status":200,"ts":1543537327796,"userAgent":"Mozilla\/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident\/6.0)","userId":"91"}


In [18]:
my_bucket = s3.Bucket('udacity-dend')
my_bucket.download_file('log_data/2018/11/2018-11-30-events.json', '2018-11-30-events.json')

In [19]:
dic_log=json.loads(line)
list(dic_log.keys())

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [20]:
filepath = '/home/anthelix/Documents/projetGit/20200217_project3_dataWarehouse/' + '2018-11-30-events.json'
filepath

'/home/anthelix/Documents/projetGit/20200217_project3_dataWarehouse/2018-11-30-events.json'

In [21]:
df_log = pd.read_json(filepath, lines=True) 
df_log[:1]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [22]:
print("df_log shape {}".format(df_log.shape))

df_log shape (388, 18)


In [23]:
print("df_log type {}".format(df_log.dtypes))

df_log type artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object


---

# Explore databases

In [24]:
%load_ext sql

In [25]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
#print(conn_string)
%sql $conn_string

'Connected: sparkifyuser@sparkifydb'

### Explore table staging_songs

In [26]:
# get numbers records in the table
query = 'SELECT COUNT(*) FROM staging_songs'
pd.read_sql(query, conn_string).style.hide_index()

count
29792


#### get length of string in staging_songs

In [27]:
query = 'SELECT MAX(num_songs) FROM staging_songs'
pd.read_sql(query, conn_string).style.hide_index()

max
1


In [28]:
query = 'SELECT COUNT(DISTINCT num_songs) FROM staging_songs'
pd.read_sql(query, conn_string).style.hide_index()

count
1


In [29]:
query = 'SELECT artist_id, LENGTH(artist_id) AS length_artist_id FROM staging_songs ORDER BY length_artist_id DESC LIMIT 1'
pd.read_sql(query, conn_string).style.hide_index()

artist_id,length_artist_id
AR73AIO1187B9AD57B,18


In [30]:
query = 'SELECT MAX(artist_latitude) AS length_artist_latitude FROM staging_songs '
pd.read_sql(query, conn_string).style.hide_index()

length_artist_latitude
69.651020


In [31]:
query = 'SELECT MAX(artist_longitude) AS length_artist_longitude FROM staging_songs '
pd.read_sql(query, conn_string).style.hide_index()

length_artist_longitude
175.471310


In [32]:
query = 'SELECT DISTINCT  artist_location, LENGTH(artist_location) AS length_artist_location FROM staging_songs WHERE artist_location IS NOT NULL ORDER BY length_artist_location DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

artist_location,length_artist_location
http://billyidol.net,176
Dubioza kolektiv (aka Dubioza as referred to by some fans) is a Bosnian band from Sarajevo,90


In [33]:
query = 'SELECT DISTINCT artist_name, LENGTH(artist_name) AS length_artist_name FROM staging_songs ORDER BY length_artist_name  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

artist_name,length_artist_name
Russell Watson / Pino Palladino / Robbie McIntosh / John Savannah / Geoff Dugmore / The Metro Voices / William Hayward / Shaun Ryder / Royal Philharmonic Orchestra / Nick Ingman,177
Yvonne S. Moriarty / Walt Fowler / Ladd McIntosh / Elizabeth Finch / Jack Smalley / Bruce Fowler / Gavin Greenaway / The Lyndhurst Orchestra / Lisa Gerrard / Hans Zimmer,169


In [34]:
query = 'SELECT DISTINCT title, LENGTH(title) AS length_title FROM staging_songs ORDER BY length_title  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

title,length_title
We're Skrewed (Otto Von Schirach My Supernatural Motorcycle Gang Will Knife U Open Piss & Shit All Over Your Organic Starbucks =iPhone Bank Account Lifeless Lifestyle Remix),173
Finally_ as that blazing sun shone down upon us_ did we know that true enemy was the voice of blind idolatry; and only then did we begin to think for ourselves.,160


In [35]:
query = 'SELECT duration, LENGTH(duration) AS length_duration FROM staging_songs ORDER BY length_duration DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

duration,length_duration
1030.190570,10
1143.039550,10


In [36]:
query = 'SELECT year, LENGTH(year) AS length_year FROM staging_songs ORDER BY length_year DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

year,length_year
2002,4
2003,4


### Explore table staging_events

In [37]:
# get number of records in the table
query = 'SELECT COUNT(*) FROM staging_events'
pd.read_sql(query, conn_string).style.hide_index()

count
8056


In [38]:
query = 'SELECT DISTINCT artist, LENGTH(artist) AS length_artist FROM staging_events WHERE artist IS NOT NULL ORDER BY length_artist DESC  LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

artist,length_artist
Emil Gilels/Orchestre de la SociÃÂ©tÃÂ© des Concerts du Conservatoire/AndrÃÂ© Cluytens,89
Manu KatchÃÂ©_ Mathias Eick_ Trygve Seim_ Marcin Wasilewski_ Slawomir Kurkiewicz,81


In [39]:
query = 'SELECT DISTINCT auth, LENGTH(auth) AS length_auth FROM staging_events ORDER BY length_auth DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

auth,length_auth
Logged Out,10
Logged In,9


In [40]:
query = 'SELECT DISTINCT firstName, LENGTH(firstName) AS length_firstName FROM staging_events WHERE firstName IS NOT NULL ORDER BY length_firstName DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

firstname,length_firstname
Jacqueline,10
Christian,9


In [41]:
query = 'SELECT DISTINCT gender, LENGTH(gender) AS length_gender FROM staging_events WHERE gender IS NOT NULL ORDER BY length_gender DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

gender,length_gender
F,1
M,1


In [42]:
query = 'SELECT itemInSession, LENGTH(itemInSession) AS length_itemInSession FROM staging_events WHERE itemInSession IS NOT NULL  ORDER BY length_itemInSession DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

iteminsession,length_iteminsession
102,3
101,3


In [43]:
query = 'SELECT DISTINCT lastName, LENGTH(lastName) AS length_lastName FROM staging_events WHERE lastName IS NOT NULL ORDER BY length_lastName  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

lastname,length_lastname
Gutierrez,9
Rodriguez,9


In [44]:
query = 'SELECT length, LENGTH(length) AS length_lengh FROM staging_events WHERE length IS NOT NULL ORDER BY length_lengh  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

length,length_lengh
1031.757910,10
2594.873020,10


In [45]:
query = 'SELECT DISTINCT level, LENGTH(level) AS length_level FROM staging_events ORDER BY length_level  DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

level,length_level
paid,4
free,4


In [46]:
query = 'SELECT DISTINCT location, LENGTH(location) AS length_location FROM staging_events WHERE location IS NOT NULL ORDER BY length_location  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

location,length_location
"Nashville-Davidson--Murfreesboro--Franklin, TN",46
"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",45


In [47]:
query = 'SELECT DISTINCT method, LENGTH(method) AS length_method FROM staging_events ORDER BY length_method  DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

method,length_method
GET,3
PUT,3


In [48]:
query = 'SELECT DISTINCT page, LENGTH(page) AS length_page FROM staging_events ORDER BY length_page  DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

page,length_page
Submit Downgrade,16
Submit Upgrade,14
Save Settings,13
Downgrade,9
NextSong,8


In [49]:
query = 'SELECT DISTINCT registration, LENGTH(registration) AS length_registration FROM staging_events WHERE registration IS NOT NULL ORDER BY length_registration  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

registration,length_registration
1541016707796.000000,13
1540306145796.000000,13


In [50]:
query = 'SELECT sessionId, LENGTH(sessionId) AS length_sessionId FROM staging_events ORDER BY length_sessionId  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

sessionid,length_sessionid
1075,4
1096,4


In [51]:
query = 'SELECT DISTINCT song, LENGTH(song) AS length_song FROM staging_events WHERE song IS NOT NULL ORDER BY length_song  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

song,length_song
Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃÂ¶sisch_ Italienisch_ Spanisch_ HollÃÂ¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch),151
The Only Difference Between Martyrdom and Suicide Is Press Coverage (Tommie Sunshine Brooklyn Fire Remix),105


In [52]:
query = 'SELECT DISTINCT status, LENGTH(status) AS length_status FROM staging_events ORDER BY length_status  DESC LIMIT 5'
pd.read_sql(query, conn_string).style.hide_index()

status,length_status
307,3
404,3
200,3


In [53]:
query = 'SELECT ts, LENGTH(ts) AS length_ts FROM staging_events ORDER BY ts  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

ts,length_ts
1543607664796,13
1543603993796,13


In [54]:
query = 'SELECT DISTINCT userAgent, LENGTH(userAgent) AS length_userAgent FROM staging_events WHERE userAgent IS NOT NULL ORDER BY length_userAgent  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

useragent,length_useragent
"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D201 Safari/9537.53""",139
"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53""",139


In [55]:
query = 'SELECT userId, LENGTH(userId) AS length_userId FROM staging_events WHERE userID IS NOT NULL ORDER BY length_userId  DESC LIMIT 2'
pd.read_sql(query, conn_string).style.hide_index()

userid,length_userid
101,3
101,3


In [56]:
query = 'SELECT COUNT(DISTINCT userId) FROM staging_events'
pd.read_sql(query, conn_string).style.hide_index()

count
97


# Queries

#### Queries to explore sparkify DataBase: "for their analytics team to continue finding insights in what songs their users are listening to."

### 1_queries to explore dimTables and Fact table

In [57]:
query = 'SELECT COUNT(*) AS Count_User FROM dimUser '
pd.read_sql(query, conn_string).style.hide_index()

count_user
105


In [58]:
query = 'SELECT Distinct COUNT(*) AS Count_Song FROM dimSong '
pd.read_sql(query, conn_string).style.hide_index()

count_song
14896


In [59]:
query = 'SELECT COUNT(*) AS Count_Artist FROM dimArtist '
pd.read_sql(query, conn_string).style.hide_index()

count_artist
10025


In [60]:
query = 'SELECT COUNT(*) AS Count_Time FROM dimTime '
pd.read_sql(query, conn_string).style.hide_index()

count_time
8023


In [61]:
query = 'SELECT COUNT(*) AS Count_Songplay FROM factSongplay '
pd.read_sql(query, conn_string).style.hide_index()

count_songplay
333


* GIVE the top 5 des chansons les plus ecoutes
* GIVE les levels et les genres des users qui ecoutent les chansons les plus ecoutes
* GIVE the top 5 des artists es plus ecoutes
* GIVE the hour where is listen

Table used for querie
dimArtist ||
dimSong   |
dimUser   |
dimTime   ||

### 2_queries for the analytic team

#### Top 5 most played artists

In [62]:
query= """
    SELECT  a.name AS ArtistName,
            sp.artist_id AS ArtistID,
            count(*) AS ArtistMostListen
    FROM factSongplay AS sp
    JOIN dimArtist AS a
    ON sp.artist_id = a.artist_id
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 5;   
"""
pd.read_sql(query, conn_string).style.hide_index()

artistname,artistid,artistmostlisten
Dwight Yoakam,AR5E44Z1187B9A1D74,37
Kid Cudi / Kanye West / Common,ARD46C811C8A414F3F,10
Kid Cudi,ARD46C811C8A414F3F,10
Lonnie Gordon,AR5EYTL1187B98EDA0,9
Ron Carter,AR37SX11187FB3E164,9


#### Top 5 most listened to songs

In [63]:
query= """
    SELECT  s.title, 
            a.name,
            count(*) AS SongMostListen
    FROM factSongplay AS sp
    JOIN dimSong AS s
    ON sp.song_id = s.song_id
    JOIN dimArtist AS a
    ON sp.artist_id = a.artist_id
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 5;
"""
pd.read_sql(query, conn_string).style.hide_index()

title,name,songmostlisten
You're The One,Dwight Yoakam,37
I CAN'T GET STARTED,Ron Carter,9
Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Lonnie Gordon,9
Nothin' On You [feat. Bruno Mars] (Album Version),B.o.B,8
Hey Daddy (Daddy's Home),Usher featuring Jermaine Dupri,6


#### The number of hours per gender and per level

In [64]:
query="""
    SELECT  sp.level, 
            u.gender,
            SUM(t.hour) AS max_TotalHour_perlevel
    FROM factSongplay AS sp
    JOIN dimUser AS u
    ON sp.user_id = u.user_id
    JOIN dimTime AS t
    ON sp.start_time=t.start_time
    GROUP BY 2, 1
    ORDER BY 3 DESC
    LIMIT 5
"""
pd.read_sql(query, conn_string).style.hide_index()

level,gender,max_totalhour_perlevel
paid,F,2793
paid,M,912
free,F,428
free,M,389


#### The average number of sessions per week per user

In [65]:
query = """
    SELECT user_id, AVG(Events) AS AVG_sessionUserWeek
    FROM (SELECT t.week AS Week,
                 sp.user_id,
                 COUNT(*) AS Events    
        FROM factSongplay AS sp
        JOIN dimTime as t
        ON sp.start_time=t.start_time
        GROUP BY 1, 2) sub
    GROUP BY user_id    
    ORDER BY AVG_sessionUserWeek DESC
    LIMIT 5;
"""
pd.read_sql(query, conn_string).style.hide_index()

user_id,avg_sessionuserweek
49,8
97,8
80,7
44,5
88,5


###  3_Explore Distkey and sortkey

####  Choose the distribution key from those columns having the highest cardinality.

In [66]:
query = """
select "column", type, encoding, distkey, sortkey 
from pg_table_def where tablename = 'dimuser';
"""
pd.read_sql(query, conn_string).style.hide_index()

column,type,encoding,distkey,sortkey
user_id,bigint,none,False,1
first_name,character varying(256),lzo,False,0
last_name,character varying(256),lzo,False,0
gender,character varying(1),lzo,False,0
level,character varying(256),lzo,False,0


In [67]:
query = """
select slice, col, num_values as rows, minvalue, maxvalue
from svv_diskusage
where name='dimuser' and col=0 and rows>0
order by slice, col;
"""
pd.read_sql(query, conn_string).style.hide_index()

slice,col,rows,minvalue,maxvalue
0,0,13,9,95
1,0,12,4,91
2,0,13,3,101
3,0,14,8,100
4,0,14,5,98
5,0,13,11,86
6,0,13,10,90
7,0,13,2,99


In [68]:
query = """
select "column", type, encoding, distkey, sortkey 
from pg_table_def where tablename = 'dimtime';
"""
pd.read_sql(query, conn_string).style.hide_index()

column,type,encoding,distkey,sortkey
start_time,timestamp without time zone,none,True,1
hour,integer,az64,False,0
day,integer,az64,False,0
week,integer,az64,False,0
month,integer,az64,False,0
year,integer,az64,False,0
weekday,character varying(256),lzo,False,0


In [69]:
query = """
select slice, col, num_values as rows, minvalue, maxvalue
from svv_diskusage
where name='dimtime' and col=0 and rows>0
order by slice, col;
"""
pd.read_sql(query, conn_string).style.hide_index()

slice,col,rows,minvalue,maxvalue
0,0,1014,594437657000000,596917894000000
1,0,1014,594421332000000,596922864000000
2,0,976,594426194000000,596918136000000
3,0,1026,594421696000000,596918119000000
4,0,992,594421306000000,596918878000000
5,0,1016,594424215000000,596919084000000
6,0,1022,594421030000000,596918676000000
7,0,963,594422253000000,596919193000000


In [70]:
query = """
select trim(name) as table, slice, sum(num_values) as rows, min(minvalue), max(maxvalue)
from svv_diskusage
where name in ('dimuser', 'dimsong', 'dimartist', 'dimtime', 'dimsongplay') 
and col =0
group by name, slice
order by name, slice;
"""
pd.read_sql(query, conn_string).style.hide_index()

table,slice,rows,min,max
dimartist,0,1253,3545266723633517121,3545833020073857601
dimartist,1,1253,3554835674579358273,3554841327413383745
dimartist,2,1253,3551423941504422465,3546106759814468161
dimartist,3,1254,3549775790754255425,3550609264208400961
dimartist,4,1254,3549472256859066945,3546948998606246465
dimartist,5,1253,3553700007883330113,3544984171310633537
dimartist,6,1252,3546381589769376321,3555949609347600961
dimartist,7,1253,3551455870291300929,3555394214258430529
dimsong,0,1862,3550034167683174227,3549212910227705683
dimsong,1,1861,3552866496751423315,3552575233965313875


In [71]:
query = """
SELECT SCHEMA schemaname,
       "table" tablename,
       table_id tableid,
       size size_in_mb,
       CASE
         WHEN diststyle NOT IN ('EVEN','ALL') THEN 1
         ELSE 0
       END has_dist_key,
       CASE
         WHEN sortkey1 IS NOT NULL THEN 1
         ELSE 0
       END has_sort_key,
       CASE
         WHEN encoded = 'Y' THEN 1
         ELSE 0
       END has_col_encoding,
       CAST(max_blocks_per_slice - min_blocks_per_slice AS FLOAT) / GREATEST(NVL (min_blocks_per_slice,0)::int,1) ratio_skew_across_slices,
       CAST(100*dist_slice AS FLOAT) /(SELECT COUNT(DISTINCT slice) FROM stv_slices) pct_slices_populated
FROM svv_table_info ti
  JOIN (SELECT tbl,
               MIN(c) min_blocks_per_slice,
               MAX(c) max_blocks_per_slice,
               COUNT(DISTINCT slice) dist_slice
        FROM (SELECT b.tbl,
                     b.slice,
                     COUNT(*) AS c
              FROM STV_BLOCKLIST b
              GROUP BY b.tbl,
                       b.slice)
        WHERE tbl IN (SELECT table_id FROM svv_table_info)
        GROUP BY tbl) iq ON iq.tbl = ti.table_id;
"""
pd.read_sql(query, conn_string).style.hide_index()


schemaname,tablename,tableid,size_in_mb,has_dist_key,has_sort_key,has_col_encoding,ratio_skew_across_slices,pct_slices_populated
public,factsongplay,100405,192,1,1,1,0.000000,100.000000
public,dimuser,100410,128,1,1,1,0.000000,100.000000
public,staging_events,100403,168,1,0,1,0.000000,100.000000
public,dimartist,100418,128,1,1,1,0.000000,100.000000
public,dimtime,100422,160,1,1,1,0.000000,100.000000
public,staging_songs,100384,104,1,0,1,0.000000,100.000000
public,dimsong,100414,128,1,1,1,0.000000,100.000000


In [72]:
query = """
SELECT
	trim(pgn.nspname) AS SCHEMA,
	trim(a.name) AS TABLE,
	id AS tableid,
	decode(pgc.reldiststyle, 0, 'even', 1, det.distkey, 8, 'all') AS distkey,
	dist_ratio.ratio::decimal (10,
		4) AS skew,
	det.head_sort AS "sortkey",
	det.n_sortkeys AS "#sks",
	b.mbytes,
	decode(b.mbytes, 0, 0, ((b.mbytes / part.total::decimal) * 100)::decimal (5, 2)) AS pct_of_total,
	decode(det.max_enc, 0, 'n', 'y') AS enc,
	a.rows,
	decode(det.n_sortkeys, 0, NULL, a.unsorted_rows) AS unsorted_rows,
	decode(det.n_sortkeys, 0, NULL, decode(a.rows, 0, 0, (a.unsorted_rows::decimal (32) / a.rows) * 100))::decimal (5,
		2) AS pct_unsorted
FROM (
	SELECT
		db_id,
		id,
		name,
		sum(ROWS) AS ROWS,
		sum(ROWS) - sum(sorted_rows) AS unsorted_rows
	FROM
		stv_tbl_perm a
	GROUP BY
		db_id,
		id,
		name) AS a
	JOIN pg_class AS pgc ON pgc.oid = a.id
	JOIN pg_namespace AS pgn ON pgn.oid = pgc.relnamespace
	LEFT OUTER JOIN (
		SELECT
			tbl,
			count(*) AS mbytes
		FROM
			stv_blocklist
		GROUP BY
			tbl) b ON a.id = b.tbl
	INNER JOIN (
		SELECT
			attrelid,
			min(
				CASE attisdistkey
				WHEN 't' THEN
					attname
				ELSE
					NULL
				END) AS "distkey",
			min(
				CASE attsortkeyord
				WHEN 1 THEN
					attname
				ELSE
					NULL
				END) AS head_sort,
			max(attsortkeyord) AS n_sortkeys,
			max(attencodingtype) AS max_enc
		FROM
			pg_attribute
		GROUP BY
			1) AS det ON det.attrelid = a.id
	INNER JOIN (
		SELECT
			tbl,
			max(mbytes)::decimal (32) / min(mbytes) AS ratio
		FROM (
			SELECT
				tbl,
				trim(name) AS name,
				slice,
				count(*) AS mbytes
			FROM
				svv_diskusage
			GROUP BY
				tbl,
				name,
				slice)
		GROUP BY
			tbl,
			name) AS dist_ratio ON a.id = dist_ratio.tbl
	JOIN (
		SELECT
			sum(capacity) AS total
		FROM
			stv_partitions
		WHERE
			part_begin = 0) AS part ON 1 = 1
WHERE
	mbytes IS NOT NULL
ORDER BY
	mbytes DESC;
"""
pd.read_sql(query, conn_string).style.hide_index()

schema,table,tableid,distkey,skew,sortkey,#sks,mbytes,pct_of_total,enc,rows,unsorted_rows,pct_unsorted
public,factsongplay,100405,start_time,1.000000,artist_id,1,192,0.020000,y,333,0.000000,0.000000
public,staging_events,100403,None,1.000000,None,0,168,0.020000,y,8056,nan,nan
public,dimtime,100422,start_time,1.000000,start_time,1,160,0.020000,y,8023,0.000000,0.000000
public,dimsong,100414,None,1.000000,song_id,1,128,0.010000,y,14896,0.000000,0.000000
public,dimartist,100418,None,1.000000,artist_id,1,128,0.010000,y,10025,0.000000,0.000000
public,dimuser,100410,None,1.000000,user_id,1,128,0.010000,y,105,0.000000,0.000000
public,staging_songs,100384,None,1.000000,None,0,104,0.010000,y,29792,nan,nan
